# AML - VOC 2012 - test set evaluation

## Imports

In [16]:
from google.colab import drive

import math
import numpy as np
import os
import shutil
import tarfile

import tensorflow as tf

In [17]:
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess_input

## Utility

In [18]:
@tf.keras.utils.register_keras_serializable()
def perfect_accuracy(y_true, y_pred, threshold=0.5):
    y_true = tf.cast(y_true, tf.int32)

    y_pred_bin = tf.cast(y_pred >= threshold, tf.int32)

    correct_predictions = tf.reduce_all(tf.equal(y_true, y_pred_bin), axis=1)

    return tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [19]:
BASELINE_IMG_SIZE = (128, 128)
RESNET50_IMG_SIZE = (224, 224)

In [20]:
ROOT_DIR = '/content/drive/MyDrive/'
BASE_DIR = ROOT_DIR + 'AML/project/test/'
#BASE_DIR = ROOT_DIR + 'project/test/'
PARTITION_FILENAMES = [f"voc_test_partition_{i}.npz" for i in range(3)]
PARTITION_PATHS = [os.path.join(BASE_DIR, filename) for filename in PARTITION_FILENAMES]

PARTITION_PATHS

['/content/drive/MyDrive/AML/project/test/voc_test_partition_0.npz',
 '/content/drive/MyDrive/AML/project/test/voc_test_partition_1.npz',
 '/content/drive/MyDrive/AML/project/test/voc_test_partition_2.npz']

In [21]:
def preprocess_images(images: np.ndarray, resize, preprocess_fn):
    resized_images = []

    for i in range(images.shape[0]):
        resized_image = tf.image.resize(images[i], resize).numpy()
        if preprocess_fn is not None:
            resized_image = preprocess_fn(resized_image)
        resized_images.append(resized_image)

    resized_images = np.array(resized_images)
    del images

    return resized_images

In [22]:
def baseline_preprocess_images(images: np.ndarray):
    return preprocess_images(images, BASELINE_IMG_SIZE, lambda x: x / 255.0)

In [23]:
def resnet_preprocess_images(images: np.ndarray):
    return preprocess_images(images, RESNET50_IMG_SIZE, resnet_preprocess_input)

In [24]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [25]:
CLASSES = [
    "aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat",
    "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]
PRED_FOLDER = "./results/VOC2012/Main"
FILE_FORMAT_PRED = "comp2_cls_test_{}.txt"
FILE_FORMAT_DET = "comp4_det_test_{}.txt"

## Predictions

In [26]:
def serialize_predictions(
    model_filepath,
    partition_paths,
    preprocess_fn,
    batch_size=1024,
    classes=CLASSES,
    pred_folder=PRED_FOLDER,
    file_format_pred=FILE_FORMAT_PRED,
    file_format_det=FILE_FORMAT_DET):
  preds = []
  filenames = []

  print(f"Loading model from '{model_filepath}'")
  model = tf.keras.models.load_model(model_filepath)

  for i, partition in enumerate(partition_paths):
    print(f"Partition {i + 1}/{len(partition_paths)}:")
    print(f"\tLoading partition...")
    partition_data = np.load(partition, allow_pickle=True)
    partition_images = partition_data['images']
    partition_filenames = partition_data['filenames']

    print(f"\tPreprocessing partition...")
    partition_images = preprocess_fn(partition_images)

    print(f"\tComputing predictions...")
    i = 0
    partition_preds_list = []
    while i < len(partition_images):
      print(f"\t\tBatch {(i // batch_size) + 1}/{math.ceil(len(partition_images) / batch_size)}")
      partition_preds_list = partition_preds_list + model.predict(partition_images[i:i+batch_size]).tolist()
      i += batch_size

    if i < len(partition_images):
      print(f"\t\tBatch {(i / batch_size) + 1}/{math.ceil(len(partition_images) // batch_size)}")
      partition_preds_list = partition_preds_list + model.predict(partition_images[i:]).tolist()

    partition_preds = np.array(partition_preds_list)
    assert partition_preds.shape == (len(partition_images), len(classes))
    del partition_data
    del partition_images
    del partition_preds_list

    preds.append(partition_preds)
    filenames.append(partition_filenames)

  print(f"Concatenating partitions predictions...")
  preds = np.concatenate(preds)
  filenames = np.concatenate(filenames)

  print(f"Sorting predictions by filename...")
  indices = np.argsort(filenames)
  preds = preds[indices]
  filenames = filenames[indices]

  print("Serializing predictions...")
  os.makedirs(pred_folder, exist_ok=True)
  preds_t = preds.T
  for i, c in enumerate(classes):
    file_path = os.path.join(pred_folder, file_format_pred.format(c))
    print(f"\tSerializing predictions for class '{c}' to '{file_path}'")
    f = open(file_path, 'w')
    for filename, pred in zip(filenames, preds_t[i]):
      f.write(f"{filename} {pred:.6f}\n")
    f.close()

  del preds

  print("Mocking results for detection task...")
  for i, c in enumerate(classes):
    file_path = os.path.join(pred_folder, file_format_det.format(c))
    print(f"\tMocking detections for class '{c}' to '{file_path}'")
    f = open(file_path, 'w')
    for filename in filenames:
      f.write(f"{filename} 0.013737 1.000000 147.000000 114.000000 242.000000\n")
    f.close()

  del filenames

In [27]:
resnet_path = "/content/drive/MyDrive/AML/project/models/model_base_resnet.keras"
resnet_filename, _ = os.path.splitext(os.path.basename(resnet_path))

resnet_path, resnet_filename

('/content/drive/MyDrive/AML/project/models/model_base_resnet.keras',
 'model_base_resnet')

In [28]:
baseline_path = "/content/drive/MyDrive/AML/project/models/baseline_augmented_model.keras"
baseline_filename, _ = os.path.splitext(os.path.basename(baseline_path))

baseline_path, baseline_filename

('/content/drive/MyDrive/AML/project/models/baseline_augmented_model.keras',
 'baseline_augmented_model')

In [29]:
model_path = resnet_path
model_filename = resnet_filename

serialize_predictions(
    baseline_path,
    PARTITION_PATHS,
    baseline_preprocess_images,
    batch_size = 2048)

Loading model from '/content/drive/MyDrive/AML/project/models/baseline_augmented_model.keras'
Partition 1/3:
	Loading partition...
	Preprocessing partition...
	Computing predictions...
		Batch 1/3
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step
		Batch 2/3
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
		Batch 3/3
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
Partition 2/3:
	Loading partition...
	Preprocessing partition...
	Computing predictions...
		Batch 1/3
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
		Batch 2/3
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
		Batch 3/3
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Partition 3/3:
	Loading partition...
	Preprocessing partition...
	Computing predictions...
		Batch 1/3
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
		Batch 2/3
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
		Batch 3/3
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Concatenating partitions predictions...
Sorting predictions by filename...
Serializing predictions...
	Serializing predictions for class 'aeroplane' to './results/VOC2012/

In [30]:
output_filename = f'results.tgz'
with tarfile.open(output_filename, "w:gz") as tar:
    tar.add(PRED_FOLDER, arcname=PRED_FOLDER)

destination = BASE_DIR + f"{model_filename}_{output_filename}"
shutil.move(output_filename, destination)

'/content/drive/MyDrive/AML/project/test/model_base_resnet_results.tgz'